In [ ]:
colab=False
if colab:
  !git clone https://github.com/cthiounn/images-generated-by-dalle-mini.git

In [ ]:
if colab:
  %run "images-generated-by-dalle-mini/util/util_extract_test1kFlickr30k.ipynb"

In [51]:
%store -r df

In [ ]:
!pip install -q transformers flax==0.3.5
!pip install -q git+https://github.com/patil-suraj/vqgan-jax.git  # VQGAN model in JAX
!pip install -q git+https://github.com/borisdayma/dalle-mini.git@26651ddd777f73dd18dade8bf543a6a8732d3669  # Model files

In [ ]:
from dalle_mini.model import CustomFlaxBartForConditionalGeneration
from transformers import BartTokenizer
import jax
import random
from tqdm.notebook import tqdm, trange
from vqgan_jax.modeling_flax_vqgan import VQModel
import numpy as np
from PIL import Image
from transformers import CLIPProcessor, FlaxCLIPModel



# make sure we use compatible versions
DALLE_REPO = "flax-community/dalle-mini"
DALLE_COMMIT_ID = "4d34126d0df8bc4a692ae933e3b902a1fa8b6114"
# make sure we use compatible versions
VQGAN_REPO = "flax-community/vqgan_f16_16384"
VQGAN_COMMIT_ID = "90cc46addd2dd8f5be21586a9a23e1b95aa506a9"
# set up model and processor
clip = FlaxCLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# set up tokenizer and model
tokenizer = BartTokenizer.from_pretrained(DALLE_REPO, revision=DALLE_COMMIT_ID)
model = CustomFlaxBartForConditionalGeneration.from_pretrained(
    DALLE_REPO, revision=DALLE_COMMIT_ID
)
vqgan = VQModel.from_pretrained(VQGAN_REPO, revision=VQGAN_COMMIT_ID)



In [ ]:
import os
import re
m= lambda s: re.findall(r'\d+',s)
root="./images-generated-by-dalle-mini/" if colab else "./" 
flickr30k_id_already_done=(set([ m(i)[0] for i in os.listdir(root+"images/flickr30k")]))


In [ ]:
for index, (imgid,first_sentence,filename) in df.iterrows():

    index=int(index)
    if str(index) not in flickr30k_id_already_done and index>500:
        # set a prompt
        prompt = first_sentence.strip()

        # tokenize the prompt
        tokenized_prompt = tokenizer(
            prompt, return_tensors="jax", padding="max_length", truncation=True, max_length=128
        )

        n_predictions = 3

        # create random keys
        seed = random.randint(0, 2 ** 32 - 1)
        key = jax.random.PRNGKey(seed)
        subkeys = jax.random.split(key, num=n_predictions)


        encoded_images = [
            model.generate(**tokenized_prompt, do_sample=True, num_beams=1, prng_key=subkey)
            for subkey in tqdm(subkeys)
        ]

        encoded_images = [img.sequences[..., 1:] for img in encoded_images]

        decoded_images = [
            vqgan.decode_code(encoded_image) for encoded_image in tqdm(encoded_images)
        ]

        clipped_images = [img.squeeze().clip(0.0, 1.0) for img in decoded_images]


        # convert to image
        images = [
            Image.fromarray(np.asarray(img * 255, dtype=np.uint8)) for img in clipped_images
        ]
        for i,j in enumerate(images):
            j.save(root+"images/flickr30k/"+str(index)+"_"+str(imgid)+"_"+str(i)+".jpeg")
    else:
        print("flickr30k_id_already_done:",index)

In [45]:
import pandas as pd
all_scores=[]
for index, (imgid,first_sentence,filename) in df.iterrows():

    # set a prompt
    prompt = first_sentence.strip()

    print(imgid)
    images=[]
    images_name=[]
    for i in os.listdir(root+"images/flickr30k"):
        if len(m(i))>1 and str(imgid)==m(i)[1]:
            images.append(Image.open(root+"images/flickr30k/"+i))
            images_name.append(i)

    # path=[ i[1] for i  in  id_and_pathfile if i[0]==int(id_coco)][0]
    # im=Image.open(path)
    # images.append(im)
    
    inputs = processor(text=prompt, images=images, return_tensors="np")
    logits = clip(**inputs).logits_per_image
    for i, l in enumerate(logits):
        all_scores.append((l[0],images_name[i]))

df_scores = pd.DataFrame(all_scores)
df_scores

25
34
51
89
97
124
133
134
137
199
219
222
228
246
259
279
304
318
340
395
434
472
487
520
539
565
615
654
661
670
734
748
754
760
828
856
887
906
917
972
973
1023
1030
1044
1072
1117
1196
1202
1206
1284
1305
1306
1310
1343
1358
1390
1401
1417
1480
1576
1594
1610
1646
1723
1771
1821
1833
1895
1981
1983
1984
1987
2044
2054
2103
2112
2113
2124
2130
2131
2139
2161
2180
2197
2260
2265
2319
2330
2410
2412
2429
2433
2458
2523
2551
2572
2600
2615
2678
2690
2788
2803
2848
2851
2906
2909
2955
2956
3008
3078
3082
3087
3097
3109
3130
3162
3245
3412
3437
3451
3459
3476
3500
3539
3648
3701
3703
3724
3771
3774
3826
3834
3951
3972
4007
4027
4028
4041
4313
4317
4343
4367
4402
4452
4501
4510
4556
4562
4663
4686
4748
4801
4816
4879
4881
4882
4902
4942
4981
5016
5056
5160
5171
5240
5261
5327
5330
5343
5441
5541
5580
5581
5595
5618
5621
5701
5724
5743
5760
5763
5767
5807
5835
5854
5867
5881
5947
5964
5970
5989
6002
6005
6008
6031
6067
6068
6078
6127
6192
6242
6250
6258
6283
6300
6306
6334
6376
6486
6532
6

Token indices sequence length is longer than the specified maximum sequence length for this model (86 > 77). Running this sequence through the model will result in indexing errors


23790


TypeError: and got incompatible shapes for broadcasting: (1, 1, 1, 86), (1, 1, 77, 77).

In [ ]:
# !nvidia-smi
# !nvcc --version
# import torch
# torch.cuda.is_available()
# # Output would be True if Pytorch is using GPU otherwise it would be False.
# import tensorflow as tf
# tf.test.gpu_device_name()
# # Standard output is '/device:GPU:0'
# !cat /usr/local/lib/python3.6/dist-packages/external/local_config_cuda/cuda/cuda/cuda_config.h | grep TF_CUDA_VERSION
# jax.devices()

In [49]:
df_scores = pd.DataFrame(all_scores)
df_scores.to_csv("minidalle_flickr30k_clip.csv")